In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip -q install findspark
!pip -q install pyspark

     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 53.1 MB/s 


In [ ]:
import os
import json
import requests
import pandas as pd
import findspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType,ArrayType,array,Dict
from pyspark.sql.functions import col, asc, desc, length,explode,map_values,udf

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
js=requests.get('https://api.hh.ru/vacancies?text=Разработчик&area=1&per_page=100&only_with_salary=True').json()['items']

In [ ]:
js[0]

{'id': '48457284',
 'premium': False,
 'name': 'Разработчик Solidity/Разработчик Rust/Разработчик Go/Разработчик python(удаленно)',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 600000, 'to': None, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2022-10-05T02:11:15+0300',
 'created_at': '2022-10-05T02:11:15+0300',
 'archived': False,
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=48457284',
 'insider_interview': None,
 'url': 'https://api.hh.ru/vacancies/48457284?host=hh.ru',
 'adv_response_url': 'https://api.hh.ru/vacancies/48457284/adv_response?host=hh.ru',
 'alternate_url': 'https://hh.ru/vacancy/48457284',
 'relations': [],
 'employer': {'id': '578702',
  'name': 'ArtVolkov.ru',
  'url': 'https://api.hh.ru/em

In [ ]:
df=pd.DataFrame(js)[['id','name','employer']].\
apply(lambda x: pd.Series((x['id'],x['name'],x['employer']['name'])),axis=1).\
rename(columns={0:'id',1:'name',2:'employer'})
df[:3]

,id,name,employer
0,48457284,Разработчик Solidity/Разработчик Rust/Разработ...,ArtVolkov.ru
1,54081874,Программист С++ Senior,Айти Прайм
2,70764477,Backend/Frontend разработчик (удаленно),ПроСММ


In [ ]:
df.dtypes

id          object
name        object
employer    object
dtype: object

In [ ]:
df.to_csv('/content/df.csv',sep=';',index=False)

In [ ]:
schema = StructType(fields=[
    StructField("id", IntegerType()),
    StructField("name", StringType ()),
    StructField("employer", StringType ()),
])

dfs = spark.read.csv('/content/df.csv', schema=schema, sep=";")

AnalysisException: ignored

In [ ]:
dfs.show(3)

+--------+--------------------+------------+
|      id|                name|    employer|
+--------+--------------------+------------+
|    null|                name|    employer|
|48457284|Разработчик Solid...|ArtVolkov.ru|
|70313760|Бэкенд разработчи...|      Рондем|
+--------+--------------------+------------+
only showing top 3 rows



In [ ]:
dfs.dtypes

[('id', 'int'), ('name', 'string'), ('employer', 'string')]

In [ ]:
dfs.head(3)

[Row(id=48457284, name='Разработчик Solidity/Разработчик Rust/Разработчик Go/Разработчик python(удаленно)', employer='ArtVolkov.ru'),
 Row(id=70313760, name='Бэкенд разработчик на языке C# (Backend-разработчик С#)', employer='Рондем'),
 Row(id=69199499, name='Ведущий full-stack разработчик', employer='Автоцентр Сити (Центр)')]

In [ ]:
dfs.collect()

In [ ]:
dfs.first()

Row(id=48457284, name='Разработчик Solidity/Разработчик Rust/Разработчик Go/Разработчик python(удаленно)', employer='ArtVolkov.ru')

In [ ]:
dfs.columns

['id', 'name', 'employer']

In [ ]:
dfs.schema

StructType([StructField('id', IntegerType(), True), StructField('name', StringType(), True), StructField('employer', StringType(), True)])

In [ ]:
dfs.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- employer: string (nullable = true)



In [ ]:
dfs = spark.read.csv('/content/df.csv', sep=";")

In [ ]:
dfs.show(3)

+--------+--------------------+------------+
|     _c0|                 _c1|         _c2|
+--------+--------------------+------------+
|      id|                name|    employer|
|48457284|Разработчик Solid...|ArtVolkov.ru|
|70313760|Бэкенд разработчи...|      Рондем|
+--------+--------------------+------------+
only showing top 3 rows



In [ ]:
dfs.dtypes

[('_c0', 'string'), ('_c1', 'string'), ('_c2', 'string')]

In [ ]:
dfs = spark.read.csv('/content/df.csv', sep=";",header=True)
dfs.dtypes

[('id', 'string'), ('name', 'string'), ('employer', 'string')]

In [ ]:
schema = StructType(fields=[
    StructField("id", IntegerType()),
    StructField("name", StringType ()),
    StructField("employer", StringType ()),
])

dfs = spark.read.csv('/content/df.csv', schema=schema, sep=";",header=True)

In [ ]:
dfs.count()

100

In [ ]:
dfs.describe().show()

+-------+-----------------+--------------------+---------+
|summary|               id|                name| employer|
+-------+-----------------+--------------------+---------+
|  count|              100|                 100|      100|
|   mean|    6.858808408E7|                null|     null|
| stddev|4637630.560776416|                null|     null|
|    min|         48457284|    .NET-разработчик|ALP GROUP|
|    max|         70530396|Фронтенд-разработчик|Элли Софт|
+-------+-----------------+--------------------+---------+



In [ ]:
dfs.dtypes

[('id', 'int'), ('name', 'string'), ('employer', 'string')]

In [ ]:
df1=pd.DataFrame(js)[['id','name','salary']]
df1[:3]

,id,name,salary
0,48457284,Разработчик Solidity/Разработчик Rust/Разработ...,"{'from': 600000, 'to': None, 'currency': 'RUR'..."
1,70313760,Бэкенд разработчик на языке C# (Backend-разраб...,"{'from': None, 'to': 400000, 'currency': 'RUR'..."
2,69199499,Ведущий full-stack разработчик,"{'from': 250000, 'to': None, 'currency': 'RUR'..."


In [ ]:
df1.to_csv('/content/df1.csv',sep=';',index=False)

In [ ]:
dfs1=spark.read.csv('/content/df1.csv',sep=';',header=True)

In [ ]:
dfs1.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)



In [ ]:
df1.to_json('/content/df1.json')

In [ ]:
with open('/content/df1.json','r') as f:
  print(f.readlines())

['{"id":{"0":"48457284","1":"70313760","2":"69199499","3":"69205889","4":"70428879","5":"69870474","6":"69442872","7":"70403895","8":"69344266","9":"54081874","10":"70465365","11":"69663974","12":"69779103","13":"70315179","14":"70514131","15":"70373340","16":"68385450","17":"69724487","18":"70423235","19":"70366507","20":"70427240","21":"54951058","22":"70407166","23":"70404612","24":"70325798","25":"70105088","26":"69452060","27":"70482424","28":"70530396","29":"70507380","30":"70010944","31":"69856407","32":"70421187","33":"69630092","34":"69877245","35":"70393250","36":"69469687","37":"55581294","38":"69873450","39":"70104695","40":"69856980","41":"70462844","42":"70427701","43":"69408727","44":"67509957","45":"70464715","46":"69709374","47":"69645775","48":"69708496","49":"69896092","50":"70156235","51":"69389024","52":"69247402","53":"69748432","54":"70405053","55":"70020738","56":"69628582","57":"70344014","58":"70506969","59":"69479136","60":"70393910","61":"69314732","62":"559

In [ ]:
df1.to_json('/content/df1.json',orient='records')
with open('/content/df1.json','r') as f:
  print(f.readlines())

['[{"id":"48457284","name":"\\u0420\\u0430\\u0437\\u0440\\u0430\\u0431\\u043e\\u0442\\u0447\\u0438\\u043a Solidity\\/\\u0420\\u0430\\u0437\\u0440\\u0430\\u0431\\u043e\\u0442\\u0447\\u0438\\u043a Rust\\/\\u0420\\u0430\\u0437\\u0440\\u0430\\u0431\\u043e\\u0442\\u0447\\u0438\\u043a Go\\/\\u0420\\u0430\\u0437\\u0440\\u0430\\u0431\\u043e\\u0442\\u0447\\u0438\\u043a python(\\u0443\\u0434\\u0430\\u043b\\u0435\\u043d\\u043d\\u043e)","salary":{"from":600000,"to":null,"currency":"RUR","gross":false}},{"id":"70313760","name":"\\u0411\\u044d\\u043a\\u0435\\u043d\\u0434 \\u0440\\u0430\\u0437\\u0440\\u0430\\u0431\\u043e\\u0442\\u0447\\u0438\\u043a \\u043d\\u0430 \\u044f\\u0437\\u044b\\u043a\\u0435 C# (Backend-\\u0440\\u0430\\u0437\\u0440\\u0430\\u0431\\u043e\\u0442\\u0447\\u0438\\u043a \\u0421#)","salary":{"from":null,"to":400000,"currency":"RUR","gross":false}},{"id":"69199499","name":"\\u0412\\u0435\\u0434\\u0443\\u0449\\u0438\\u0439 full-stack \\u0440\\u0430\\u0437\\u0440\\u0430\\u0431\\u043e\\u0

In [ ]:
dfs2=spark.read.json('/content/df1.json',)
dfs2.show()

+--------+--------------------+--------------------+
|      id|                name|              salary|
+--------+--------------------+--------------------+
|48457284|Разработчик Solid...|{RUR, 600000, fal...|
|70313760|Бэкенд разработчи...|{RUR, null, false...|
|69199499|Ведущий full-stac...|{RUR, 250000, tru...|
|69205889|     Разработчик C++|{RUR, 300000, fal...|
|70428879|    Разработчик .NET|{RUR, 180000, fal...|
|69870474|Software engineer...|{USD, 5000, false...|
|69442872|Разработчик full-...|{USD, 7000, false...|
|70403895|    Программист .NET|{RUR, 150000, fal...|
|69344266|Junior fullstack ...|{RUR, 90000, fals...|
|54081874|Программист С++ S...|{RUR, 400000, fal...|
|70465365|Fullstack Develop...|{EUR, null, true,...|
|69663974|     РНР разработчик|{RUR, 150000, fal...|
|69779103| Backend разработчик|{USD, 2000, false...|
|70315179|Front-End разрабо...|{RUR, null, false...|
|70514131|Backend Developer Qt|{RUR, 150000, fal...|
|70373340|         Программист|{RUR, 200000, t

In [ ]:
dfs2.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: struct (nullable = true)
 |    |-- currency: string (nullable = true)
 |    |-- from: long (nullable = true)
 |    |-- gross: boolean (nullable = true)
 |    |-- to: long (nullable = true)



In [ ]:
schema = StructType(fields=[
    StructField('id',IntegerType()),
    StructField('name',StringType()),
    StructField('salary',ArrayType(StringType()))])

In [ ]:
dfs2=spark.read.json('/content/df1.json')

In [ ]:
dfs3=dfs2.rdd.map(lambda x:x[2][1]).toDF(StructType(fields=[StructField('from_sal',StringType())]))
#dfs3.show()

In [ ]:
dfs3

Py4JJavaError: ignored

In [ ]:
dfs2.withColumn('sal',dfs2.rdd.map(lambda x:x[2][1]).c)

TypeError: ignored

In [ ]:
dfs2.select(col('id'),explode(col('salary')))

AnalysisException: ignored

In [ ]:
dfs2=spark.read.json('/content/df1.json',multiLine=True,)
dfs2.show()

+--------------------+--------------------+--------------------+
|                  id|                name|              salary|
+--------------------+--------------------+--------------------+
|{48457284, 703137...|{Разработчик Soli...|{{RUR, 600000, fa...|
+--------------------+--------------------+--------------------+



In [ ]:
dfs.groupby('employer').count().sort('count',ascending=False).show(5)

+--------------------+-----+
|            employer|count|
+--------------------+-----+
|            HR Prime|    3|
|              amoCRM|    3|
|        Hi, Rockits!|    2|
|              Рондем|    2|
|Федеральное автон...|    2|
+--------------------+-----+
only showing top 5 rows



In [ ]:
dfs.select('employer','id').show(5)

+--------------------+--------+
|            employer|      id|
+--------------------+--------+
|        ArtVolkov.ru|48457284|
|              Рондем|70313760|
|Автоцентр Сити (Ц...|69199499|
|           R2CMarket|69205889|
|             Scalaxi|70428879|
+--------------------+--------+
only showing top 5 rows



In [ ]:
dfs.select(col('employer').alias('работодатель')).show(3)

+--------------------+
|        работодатель|
+--------------------+
|        ArtVolkov.ru|
|              Рондем|
|Автоцентр Сити (Ц...|
+--------------------+
only showing top 3 rows



In [ ]:
dfs.filter(col('id') > 70313760).show(3)

+--------+--------------------+--------+
|      id|                name|employer|
+--------+--------------------+--------+
|70428879|    Разработчик .NET| Scalaxi|
|70403895|    Программист .NET|МЕТРАФОР|
|70465365|Fullstack Develop...|HR Prime|
+--------+--------------------+--------+
only showing top 3 rows



In [ ]:
dfs.orderBy(col('employer').desc(), col('name').asc()).show(3)

+--------+--------------------+--------------------+
|      id|                name|            employer|
+--------+--------------------+--------------------+
|69877245|JavaScript разраб...|           Элли Софт|
|69856980|  Golang разработчик|       ШтрафовНет.ру|
|70331198|    Программист Java|Шмыков Дмитрий Вя...|
+--------+--------------------+--------------------+
only showing top 3 rows



In [ ]:
dfs.schema

StructType([StructField('id', IntegerType(), True), StructField('name', StringType(), True), StructField('employer', StringType(), True)])

In [ ]:
dfs.dtypes

[('id', 'int'), ('name', 'string'), ('employer', 'string')]

In [ ]:
dfs.take(4)

[Row(id=48457284, name='Разработчик Solidity/Разработчик Rust/Разработчик Go/Разработчик python(удаленно)', employer='ArtVolkov.ru'),
 Row(id=70313760, name='Бэкенд разработчик на языке C# (Backend-разработчик С#)', employer='Рондем'),
 Row(id=69199499, name='Ведущий full-stack разработчик', employer='Автоцентр Сити (Центр)'),
 Row(id=69205889, name='Разработчик C++', employer='R2CMarket')]

In [ ]:
dfs.distinct().show(3)

+--------+--------------------+----------------+
|      id|                name|        employer|
+--------+--------------------+----------------+
|70105088|Junior frontend р...|    Лотус студио|
|70427701| Node.js разработчик|Челлендж - Групп|
|69034150|PHP backend разра...|          amoCRM|
+--------+--------------------+----------------+
only showing top 3 rows



In [ ]:
dfs.na.drop().show(2)

+--------+--------------------+------------+
|      id|                name|    employer|
+--------+--------------------+------------+
|48457284|Разработчик Solid...|ArtVolkov.ru|
|70313760|Бэкенд разработчи...|      Рондем|
+--------+--------------------+------------+
only showing top 2 rows



In [ ]:
dfs.na.fill(dfs.select(f.mean(dfs['id'])).collect()[0][0]).show(2)

+--------+--------------------+------------+
|      id|                name|    employer|
+--------+--------------------+------------+
|48457284|Разработчик Solid...|ArtVolkov.ru|
|70313760|Бэкенд разработчи...|      Рондем|
+--------+--------------------+------------+
only showing top 2 rows



In [ ]:
dfs.na.replace(0, 1).show(2)

+--------+--------------------+------------+
|      id|                name|    employer|
+--------+--------------------+------------+
|48457284|Разработчик Solid...|ArtVolkov.ru|
|70313760|Бэкенд разработчи...|      Рондем|
+--------+--------------------+------------+
only showing top 2 rows



In [ ]:
dfs.select('name', f.when(dfs.id>= 60000000, 1).otherwise(0)).show(5)

+--------------------+--------------------------------------------+
|                name|CASE WHEN (id >= 60000000) THEN 1 ELSE 0 END|
+--------------------+--------------------------------------------+
|Разработчик Solid...|                                           0|
|Бэкенд разработчи...|                                           1|
|Ведущий full-stac...|                                           1|
|     Разработчик C++|                                           1|
|    Разработчик .NET|                                           1|
+--------------------+--------------------------------------------+
only showing top 5 rows

